# Cleaning Dirty and Messy Data with Python

In this exercise, you will solve a few problems applying the methods you learned to clean dirty and messy data with Python. 

We will be considering a small hypothetical dataset of time series data being reported from pressure sensors on a manufacturing floor (that was post-processed) below.

In [1]:
#Imports - DO NOT MODIFY
import pandas as pd
import numpy as np

## Dataset Context

Details of the data:

- The sensor location indicates the location where the sensor is placed and the system (A, B) it corresponds to (e.g. Entry A).
- The dates indicates the days where data was collected from the sensors (e.g. 1/5/2023).
- The numeric values indicate the pressure sensors' measurements (ranging from 0-25 bar)

In [2]:
#FILL IN - Read in pressure_sensor_data.tsv into a pandas dataframe
df = pd.read_csv('pressure_sensor_data.tsv', sep='\t')

In [3]:
df.head()

,Sensor Location,1/5/2023,1/6/2023,1/18/2023,1/28/2023,2/6/2023,2/7/2023,2/9/2023,2/28/2023
0,Entry A,29.0,37.0,NaN,NaN,23.0,16.0,13.0,NaN
1,Area A,0.0,11.0,14.0,2.0,11.0,4.0,15.0,15.0
2,Area B,NaN,28.0,0.0,21.0,19.0,23.0,6.0,16.0
3,Entry A,6.0,NaN,4.0,5.0,NaN,NaN,11.0,3.0
4,Exit A,33.0,7.0,2.0,22.0,23.0,11.0,NaN,13.0


## 1.  Tidiness Issue:  Column Headers are values, not variable names

Use `pd.melt()` to ensure the column headers are no longer values and are instead variable names. Ensure the columns are named informatively when calling `pd.melt()` based on the values they represent.

Your result should have three variables (columns): `Sensor Location`, `Date`, `Pressure`.

In [4]:
##FILL IN  - use .melt() to solve the tidiness issue
updated_df = df.melt(id_vars='Sensor Location', var_name='Date', value_name='Pressure')
#Show the first few rows of the dataframe
updated_df.head()

,Sensor Location,Date,Pressure
0,Entry A,1/5/2023,29.0
1,Area A,1/5/2023,0.0
2,Area B,1/5/2023,NaN
3,Entry A,1/5/2023,6.0
4,Exit A,1/5/2023,33.0


## 2. Tidiness Issue: Multiple variables stored in one column

The `Sensor Location` column violates one of the guidelines of tidiness - multiple variables being stored in column. Use the updated data from step 1 to identify why and resolve thie issue.

*Hint:* Consider using the `str.split(expand=True)` functionality since the sensor location and system data is separated by a space (e.g., "Entry A")

*FILL IN - how are multiple variables being stored in the `Sensor Location` column?*

The column includes both the sensor location and the system it corresponds to. This information can be seperated into two columns.

In [5]:
# FILL IN - split the variables in the Sensor Location into multiple variables/columns
updated_df[['Sensor Location', 'System']] = updated_df['Sensor Location'].str.split(expand=True)
updated_df.head()

,Sensor Location,Date,Pressure,System
0,Entry,1/5/2023,29.0,A
1,Area,1/5/2023,0.0,A
2,Area,1/5/2023,NaN,B
3,Entry,1/5/2023,6.0,A
4,Exit,1/5/2023,33.0,A


## 3. Quality Issue: Validity and Accuracy

3.1 Between Jan 5th to Jan 6th, some of sensors' data for both Systems A and B were found to be corruputed due to a sudden, very high increase in temperature. 

Filter out rows where the sensors' reported data is **greater than 25 bar** into a seperate dataframe, and drop these rows from the original dataframe. 

*Hint:* Specify the invalid rows' index to drop these rows from the original dataframe.

In [6]:
# FILL IN - get the rows where the data is greater than 25 bar in a seperate dataframe
invalid_data = updated_df[updated_df['Pressure'] > 25]
#Print the index of the new dataframe with invalid rows
invalid_data.index

Int64Index([0, 4, 6, 8], dtype='int64')

In [7]:
#FILL IN - drop the invalid rows from the original dataframe using .drop(...)
updated_df = updated_df.drop(invalid_data.index)
#Reset the index
updated_df = updated_df.reset_index()
#Display the first few rows
updated_df.head()

,index,Sensor Location,Date,Pressure,System
0,1,Area,1/5/2023,0.0,A
1,2,Area,1/5/2023,NaN,B
2,3,Entry,1/5/2023,6.0,A
3,5,Exit,1/5/2023,18.0,B
4,7,Area,1/6/2023,11.0,A


3.2 Use the `describe` function to check that the fix was successful. Are the `min` and `max` of the `Pressure` variable in the right range?

In [8]:
#FILL IN - validate the range using .describe()
updated_df.describe()

,index,Pressure
count,44.000000,36.000000
mean,25.227273,11.583333
std,13.296028,6.983143
min,1.000000,0.000000
25%,14.750000,5.000000
50%,25.500000,12.000000
75%,36.250000,16.000000
max,47.000000,23.000000


*FILL IN - is the range correct (Yes/No)*: ...

The `Pressure` variable's range is correct, within 0-25 bar.